In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

#이미지가 저장될 폴더가 없다면 만듭니다.
import os
if not os.path.exists("./gan_images"):
    os.makedirs("./gan_images")

np.random.seed(3)
tf.random.set_seed(3)

In [4]:
#생성자 모델을 만듭니다.
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))
                                # 음수가 표현이 되는 LeakyReLU(음수 기울기 0.2)
generator.add(BatchNormalization())
                    # BatchNormalization()  평균 0 분산이 1이 되도록 재배치.
generator.add(Reshape((7, 7, 128)))
generator.add(UpSampling2D())  #UpSampling2D() 이미지크기를 2배로 늘림
generator.add(Conv2D(64, kernel_size=5, padding='same' )  )
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D())
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))

In [5]:
#판별자 모델을 만듭니다.
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2,
                         input_shape=(28,28,1), padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2,
                                             padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
                            # 이미가 진짜인지 거짓인지 판별하는 이진 분류
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable = False

In [6]:
#생성자와 판별자 모델을 연결시키는 gan 모델을 만듭니다.
ginput = Input(shape=(100,))
# 생성자와 판별자 모델을 연결(생성자가 생성한 이미지를 판별자에게 전달)
dis_output = discriminator(generator(ginput))

gan = Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 28, 28, 1)         865281    
                                                                 
 sequential_1 (Sequential)   (None, 1)                 212865    
                                                                 
Total params: 1078146 (4.11 MB)
Trainable params: 852609 (3.25 MB)
Non-trainable params: 225537 (881.00 KB)
_________________________________________________________________


In [ ]:
#신경망을 실행시키는 함수를 만듭니다.
def gan_train(epoch, batch_size, saving_interval):
    # MNIST 데이터 불러오기
    (X_train, _), (_, _) = mnist.load_data()
    # 앞서 불러온 적 있는 MNIST를 다시 이용합니다. 단, 테스트과정은 필요없고
    # 이미지만 사용할 것이기 때문에 X_train만 불러왔습니다.

    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
    X_train = (X_train - 127.5) / 127.5
    # 픽셀값은 0에서 255사이의 값입니다.
    # 이전에 255로 나누어 줄때는 이를 0~1사이의 값으로 바꾸었던 것인데,
    # 여기서는 127.5를 빼준 뒤 127.5로 나누어 줌으로 인해
    # -1에서 1사이의 값으로 바뀌게 됩니다.

    true = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for i in range(epoch):
        # 실제 데이터를 판별자에 입력하는 부분입니다.
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        d_loss_real = discriminator.train_on_batch(imgs,true)
                                         #진짜 데이터는 true 로 학습

        #가상 이미지를 판별자에 입력하는 부분입니다.
        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
                                           #가짜 데이터는 false로 학습

        #판별자 오차를 계산합니다. (오차 평균)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        #생성자의 오차를 계산합니다.
        g_loss = gan.train_on_batch(noise, true)
        # noise로 만든 가짜이미지를 true로 학습 (판별자 속이기)
        # g_loss가 작을 수록 진짜같은 이미지를 생성하게 되었다는 의미.

        print('epoch:%d'%i,'d_loss:%.4f'%d_loss,' g_loss:%.4f'% g_loss)

        # 이부분은 중간 과정(generator가 생성한) 이미지로 저장해 주는 부분입니다.
        # 본 장의 주요 내용과 관련이 없어 소스코드만 첨부합니다.
        # 만들어진 이미지들은 gan_images 폴더에 저장됩니다.
        if i % saving_interval == 0:
              #r, c = 5, 5
            noise = np.random.normal(0, 1, (25, 100))
            gen_imgs = generator.predict(noise)

              # Rescale images 0 - 1
            gen_imgs = 0.5 * gen_imgs + 0.5

            fig, axs = plt.subplots(5, 5)
            count = 0
            for j in range(5):
                  for k in range(5):
                    axs[j, k].imshow(gen_imgs[count, :, :, 0], cmap='gray')
                    axs[j, k].axis('off')
                    count += 1
            fig.savefig("gan_images/gan_mnist_%d.png" % i)

gan_train(8001, 32, 200)
# 4000번 반복되고(+1을 해 주는 것에 주의), 배치 사이즈는 32,
# 200번 마다 결과가 저장되게 하였습니다.
# 학습 후반부로 갈 수로 generator는 점점 더 진짜 같은 이미지를 생성해줍니다.

1/1 [==============================] - 0s 47ms/step
epoch:0 d_loss:0.3881  g_loss:0.5963
1/1 [==============================] - 0s 34ms/step
epoch:1 d_loss:0.3400  g_loss:0.0940
1/1 [==============================] - 0s 32ms/step
epoch:2 d_loss:0.6316  g_loss:0.0196
1/1 [==============================] - 0s 42ms/step
epoch:3 d_loss:0.7949  g_loss:0.0149
1/1 [==============================] - 0s 50ms/step
epoch:4 d_loss:0.7332  g_loss:0.0650
1/1 [==============================] - 0s 33ms/step
epoch:5 d_loss:0.5251  g_loss:0.3371
1/1 [==============================] - 0s 43ms/step
epoch:6 d_loss:0.4874  g_loss:0.6960
1/1 [==============================] - 0s 49ms/step
epoch:7 d_loss:0.5671  g_loss:0.7654
1/1 [==============================] - 0s 47ms/step
epoch:8 d_loss:0.5393  g_loss:0.6533
1/1 [==============================] - 0s 29ms/step
epoch:9 d_loss:0.5143  g_loss:0.6172
1/1 [==============================] - 0s 41ms/step
epoch:10 d_loss:0.4814  g_loss:0.5907
1/1 [===============

epoch:90 d_loss:0.4705  g_loss:2.2504
1/1 [==============================] - 0s 43ms/step
epoch:91 d_loss:0.3705  g_loss:3.0361
1/1 [==============================] - 0s 51ms/step
epoch:92 d_loss:0.5225  g_loss:2.7377
1/1 [==============================] - 0s 31ms/step
epoch:93 d_loss:0.7752  g_loss:3.0091
1/1 [==============================] - 0s 38ms/step
epoch:94 d_loss:1.0685  g_loss:2.0348
1/1 [==============================] - 0s 36ms/step
epoch:95 d_loss:0.9124  g_loss:1.8730
1/1 [==============================] - 0s 27ms/step
epoch:96 d_loss:0.5063  g_loss:1.7100
1/1 [==============================] - 0s 38ms/step
epoch:97 d_loss:0.7287  g_loss:2.0312
1/1 [==============================] - 0s 33ms/step
epoch:98 d_loss:0.7806  g_loss:1.8221
1/1 [==============================] - 0s 45ms/step
epoch:99 d_loss:0.7569  g_loss:1.1104
1/1 [==============================] - 0s 40ms/step
epoch:100 d_loss:0.5324  g_loss:0.7685
1/1 [==============================] - 0s 50ms/step
epoch:101

1/1 [==============================] - 0s 31ms/step
epoch:181 d_loss:0.5917  g_loss:1.9307
1/1 [==============================] - 0s 29ms/step
epoch:182 d_loss:0.6877  g_loss:1.8228
1/1 [==============================] - 0s 30ms/step
epoch:183 d_loss:0.4730  g_loss:2.0538
1/1 [==============================] - 0s 43ms/step
epoch:184 d_loss:0.7699  g_loss:1.8290
1/1 [==============================] - 0s 45ms/step
epoch:185 d_loss:0.5080  g_loss:2.1649
1/1 [==============================] - 0s 34ms/step
epoch:186 d_loss:0.4494  g_loss:2.2051
1/1 [==============================] - 0s 45ms/step
epoch:187 d_loss:0.5386  g_loss:2.1992
1/1 [==============================] - 0s 41ms/step
epoch:188 d_loss:0.5185  g_loss:2.1247
1/1 [==============================] - 0s 48ms/step
epoch:189 d_loss:0.4825  g_loss:2.3526
1/1 [==============================] - 0s 42ms/step
epoch:190 d_loss:0.5317  g_loss:2.4064
1/1 [==============================] - 0s 51ms/step
epoch:191 d_loss:0.5790  g_loss:2.2552

1/1 [==============================] - 0s 43ms/step
epoch:270 d_loss:0.6031  g_loss:1.7866
1/1 [==============================] - 0s 44ms/step
epoch:271 d_loss:0.4384  g_loss:1.7664
1/1 [==============================] - 0s 45ms/step
epoch:272 d_loss:0.5275  g_loss:1.8053
1/1 [==============================] - 0s 45ms/step
epoch:273 d_loss:0.5813  g_loss:1.9648
1/1 [==============================] - 0s 44ms/step
epoch:274 d_loss:0.5719  g_loss:2.1411
1/1 [==============================] - 0s 44ms/step
epoch:275 d_loss:0.6354  g_loss:1.9712
1/1 [==============================] - 0s 43ms/step
epoch:276 d_loss:0.6854  g_loss:2.0224
1/1 [==============================] - 0s 48ms/step
epoch:277 d_loss:0.5588  g_loss:2.6409
1/1 [==============================] - 0s 36ms/step
epoch:278 d_loss:0.6479  g_loss:2.0356
1/1 [==============================] - 0s 47ms/step
epoch:279 d_loss:0.4605  g_loss:2.2155
1/1 [==============================] - 0s 34ms/step
epoch:280 d_loss:0.5140  g_loss:1.7831

1/1 [==============================] - 0s 30ms/step
epoch:360 d_loss:0.4558  g_loss:2.1944
1/1 [==============================] - 0s 28ms/step
epoch:361 d_loss:0.3975  g_loss:2.3322
1/1 [==============================] - 0s 44ms/step
epoch:362 d_loss:0.6423  g_loss:3.0469
1/1 [==============================] - 0s 44ms/step
epoch:363 d_loss:0.3019  g_loss:3.3170
1/1 [==============================] - 0s 45ms/step
epoch:364 d_loss:0.4511  g_loss:3.1278
1/1 [==============================] - 0s 45ms/step
epoch:365 d_loss:0.4710  g_loss:2.4850
1/1 [==============================] - 0s 44ms/step
epoch:366 d_loss:0.6129  g_loss:1.8346
1/1 [==============================] - 0s 45ms/step
epoch:367 d_loss:0.4507  g_loss:2.4394
1/1 [==============================] - 0s 44ms/step
epoch:368 d_loss:0.4921  g_loss:2.2801
1/1 [==============================] - 0s 29ms/step
epoch:369 d_loss:0.3416  g_loss:3.6574
1/1 [==============================] - 0s 43ms/step
epoch:370 d_loss:0.5954  g_loss:3.2409

...

1/1 [==============================] - 0s 29ms/step
epoch:2759 d_loss:0.3547  g_loss:1.8929
1/1 [==============================] - 0s 27ms/step
epoch:2760 d_loss:0.5462  g_loss:1.5829
1/1 [==============================] - 0s 44ms/step
epoch:2761 d_loss:0.4835  g_loss:1.6638
1/1 [==============================] - 0s 28ms/step
epoch:2762 d_loss:0.5471  g_loss:1.4368
1/1 [==============================] - 0s 26ms/step
epoch:2763 d_loss:0.5066  g_loss:1.2402
1/1 [==============================] - 0s 28ms/step
epoch:2764 d_loss:0.5765  g_loss:1.5495
1/1 [==============================] - 0s 43ms/step
epoch:2765 d_loss:0.4314  g_loss:1.3649
1/1 [==============================] - 0s 44ms/step
epoch:2766 d_loss:0.5345  g_loss:1.6373
1/1 [==============================] - 0s 44ms/step
epoch:2767 d_loss:0.5630  g_loss:1.4882
1/1 [==============================] - 0s 44ms/step
epoch:2768 d_loss:0.5511  g_loss:1.4223
1/1 [==============================] - 0s 41ms/step
epoch:2769 d_loss:0.7468  g_

1/1 [==============================] - 0s 30ms/step
epoch:2847 d_loss:0.6456  g_loss:1.1435
1/1 [==============================] - 0s 41ms/step
epoch:2848 d_loss:0.5425  g_loss:1.7765
1/1 [==============================] - 0s 42ms/step
epoch:2849 d_loss:0.5322  g_loss:1.4019
1/1 [==============================] - 0s 43ms/step
epoch:2850 d_loss:0.5683  g_loss:1.4357
1/1 [==============================] - 0s 45ms/step
epoch:2851 d_loss:0.5758  g_loss:1.0194
1/1 [==============================] - 0s 44ms/step
epoch:2852 d_loss:0.5228  g_loss:1.5003
1/1 [==============================] - 0s 45ms/step
epoch:2853 d_loss:0.5067  g_loss:1.4882
1/1 [==============================] - 0s 44ms/step
epoch:2854 d_loss:0.5723  g_loss:1.2079
1/1 [==============================] - 0s 44ms/step
epoch:2855 d_loss:0.4324  g_loss:1.5938
1/1 [==============================] - 0s 44ms/step
epoch:2856 d_loss:0.4654  g_loss:1.3555
1/1 [==============================] - 0s 44ms/step
epoch:2857 d_loss:0.5072  g_

1/1 [==============================] - 0s 30ms/step
epoch:2936 d_loss:0.5415  g_loss:1.7202
1/1 [==============================] - 0s 42ms/step
epoch:2937 d_loss:0.4046  g_loss:1.6307
1/1 [==============================] - 0s 44ms/step
epoch:2938 d_loss:0.5638  g_loss:1.4211
1/1 [==============================] - 0s 44ms/step
epoch:2939 d_loss:0.5247  g_loss:1.1433
1/1 [==============================] - 0s 45ms/step
epoch:2940 d_loss:0.4564  g_loss:1.5800
1/1 [==============================] - 0s 44ms/step
epoch:2941 d_loss:0.3351  g_loss:1.6579
1/1 [==============================] - 0s 45ms/step
epoch:2942 d_loss:0.5000  g_loss:1.6217
1/1 [==============================] - 0s 45ms/step
epoch:2943 d_loss:0.5694  g_loss:1.4855
1/1 [==============================] - 0s 47ms/step
epoch:2944 d_loss:0.5285  g_loss:1.5225
1/1 [==============================] - 0s 47ms/step
epoch:2945 d_loss:0.4973  g_loss:1.6234
1/1 [==============================] - 0s 45ms/step
epoch:2946 d_loss:0.5344  g_

1/1 [==============================] - 0s 53ms/step
epoch:3024 d_loss:0.5330  g_loss:2.0270
1/1 [==============================] - 0s 23ms/step
epoch:3025 d_loss:0.4362  g_loss:1.6114
1/1 [==============================] - 0s 37ms/step
epoch:3026 d_loss:0.3140  g_loss:1.7135
1/1 [==============================] - 0s 32ms/step
epoch:3027 d_loss:0.4434  g_loss:1.8080
1/1 [==============================] - 0s 43ms/step
epoch:3028 d_loss:0.2930  g_loss:1.4317
1/1 [==============================] - 0s 34ms/step
epoch:3029 d_loss:0.4883  g_loss:1.7485
1/1 [==============================] - 0s 50ms/step
epoch:3030 d_loss:0.4446  g_loss:1.3488
1/1 [==============================] - 0s 35ms/step
epoch:3031 d_loss:0.4528  g_loss:1.7757
1/1 [==============================] - 0s 40ms/step
epoch:3032 d_loss:0.6167  g_loss:1.9366
1/1 [==============================] - 0s 44ms/step
epoch:3033 d_loss:0.5245  g_loss:2.0120
1/1 [==============================] - 0s 40ms/step
epoch:3034 d_loss:0.6063  g_

1/1 [==============================] - 0s 48ms/step
epoch:3113 d_loss:0.4279  g_loss:1.4937
1/1 [==============================] - 0s 50ms/step
epoch:3114 d_loss:0.4171  g_loss:1.7706
1/1 [==============================] - 0s 35ms/step
epoch:3115 d_loss:0.5213  g_loss:1.7603
1/1 [==============================] - 0s 25ms/step
epoch:3116 d_loss:0.6298  g_loss:1.8493
1/1 [==============================] - 0s 51ms/step
epoch:3117 d_loss:0.5467  g_loss:1.4585
1/1 [==============================] - 0s 33ms/step
epoch:3118 d_loss:0.4535  g_loss:1.3144
1/1 [==============================] - 0s 32ms/step
epoch:3119 d_loss:0.4644  g_loss:1.6329
1/1 [==============================] - 0s 38ms/step
epoch:3120 d_loss:0.4113  g_loss:2.1504
1/1 [==============================] - 0s 41ms/step
epoch:3121 d_loss:0.6196  g_loss:1.8193
1/1 [==============================] - 0s 33ms/step
epoch:3122 d_loss:0.5291  g_loss:1.5644
1/1 [==============================] - 0s 34ms/step
epoch:3123 d_loss:0.5457  g_